In [ ]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://www-us.apache.org/dist/spark/spark-2.3.4/spark-2.3.4-bin-hadoop2.7.tgz
#!tar xf spark-2.3.4-bin-hadoop2.7.tgz

#!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

# 1. Lectura de datos

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!ls '/content/drive/My Drive/PySpark Machine Learning en plataforma Big Data/Training R ML en Casos de Negocios/1_caso negocio 1 vtas seguros/0. DS_Seguros_Salud.csv'

In [ ]:
ls -la

In [ ]:
pwd

In [ ]:
!free -h


In [ ]:
!df -h

In [ ]:
!apt install tree

In [ ]:
!tree

# 2. Carga de datos en formato spark

In [ ]:
! ls /content/drive/My\ Drive | grep -i py

In [ ]:
DS_Salud = spark.read.csv(
    '/content/drive/My Drive/PySpark Machine Learning en plataforma Big Data/Training R ML en Casos de Negocios/1_caso negocio 1 vtas seguros/0. DS_Seguros_Salud.csv', 
    sep=';', 
    header=True, 
    inferSchema=True
)

# 2.1 Revision de formatos 
DS_Salud.printSchema()

In [ ]:
!df -h

In [ ]:
!free -h

In [ ]:
# Importando desde una tabla en ls

#DS_Salud = spark.read.csv('0. DS_Seguros_Salud.csv', sep=';', header=True, inferSchema=True)

# 2.1 Revision de formatos 
#DS_Salud.printSchema()

In [ ]:
DS_Salud.describe().show()

# 2.1 Exploracion del DataDrame


## 2.1.1 Exploracion - Columnas y dimension de Dataframe

In [ ]:
# Nombres de columnas
DS_Salud.columns

In [ ]:
# Dimensionamiento del Dataframe
print(DS_Salud.count(),len(DS_Salud.columns))

## 2.1.2 Operaciones con el Dataframe 

In [ ]:
# Agregando columna en formato de anios
DS_Salud = DS_Salud.withColumn(
    "ANTIGUEDAD_ANIOS",
    (DS_Salud["ANTIGUEDAD_MES"]/12)
)
DS_Salud.show(10, False)

In [ ]:
# Agrupamiento por Departamento de residencia
DS_Salud.groupBy('DEPARTAMENTO').count().show(15)

In [ ]:
# Agrupamiento por Departamento de residencia
DS_Salud.groupBy('DEPARTAMENTO').mean('VENTA_SEGURO').show(15)

In [ ]:
DS_Salud.groupBy('DEPARTAMENTO').max('VENTA_SEGURO').show(15)

In [ ]:
print(dir(DS_Salud.groupBy('DEPARTAMENTO')))

In [ ]:
DS_Salud.groupBy('DEPARTAMENTO').agg({
    'DEPARTAMENTO': 'min', 'VENTA_SEGURO': 'avg', 
}).show(15)

In [ ]:
# Filtrado de Dataframe
DS_Salud.filter(DS_Salud['DEPARTAMENTO']=='LA LIBERTAD').groupBy('VENTA_SEGURO').count().show()

In [ ]:
# Filtrado de Dataframe y seleccionadono
DS_Salud.filter(DS_Salud['DEPARTAMENTO']=='AREQUIPA').select('VENTA_SEGURO','DEPARTAMENTO').show()

In [ ]:
# Doble filtro y conteo

DS_Salud.filter((DS_Salud['VENTA_SEGURO']=='1')&(DS_Salud['ANTIGUEDAD_ANIOS'] >5)).count()

#DS_Salud.columns
#DS_Salud.filter((DS_Salud['VENTA_SEGURO']=='1')&(DS_Salud['ANTIGUEDAD_MES'] >5)).count()

In [ ]:
# Nro de tarjetas de credito unicas

DS_Salud.select('VENTA_SEGURO','DEPARTAMENTO').show()


In [ ]:
DS_Salud.groupBy('VENTA_SEGURO').mean().show(5)

In [ ]:
DS_Salud.describe().show()

In [ ]:
help(DS_Salud.describe)

In [ ]:
#2.2 Lectura de 5 filas Spark

DS_Salud.head(5)

In [ ]:
#2.3 Lectura de 5 filas tabla

#DS_Salud.show(5)
DS_Salud.columns

In [ ]:
#2.4 Conteo de registros

#DS_Salud.count()

#2.5 Numero de columnas

len(DS_Salud.columns)

# 3. Exploracion de valores

In [ ]:
# seleccionando 5  clientes
DS_Salud.select('VENTA_SEGURO','monto_ope_t2').show(5)

In [ ]:
# Clientes con caracteristicas diferentes
#DS_Salud.select('VENTA_SEGURO','NUM_TC_TOTAL_T2').distinct().show()
DS_Salud.select('VENTA_SEGURO','NUM_TC_TOTAL_T2').distinct().count()

In [ ]:
# Filtrado del data set
DS_Salud.filter(DS_Salud.monto_ope_t2>548)

# 4. Resumen descriptivo de tabla

In [ ]:
DS_Salud.describe().show()

In [ ]:
#DS_Salud.describe('VENTA_SEGURO').show()

In [ ]:
# Tabla de frecuencias cruzadas

DS_Salud.crosstab('VENTA_SEGURO','SEXO').show()

In [ ]:
DS_Salud.crosstab('VENTA_SEGURO','DEPARTAMENTO').show()

# 5. Analisis de relevancia del cliente automatizada (Information Value)

In [ ]:
!pip install scorecardpy

In [ ]:
import scorecardpy as sc
import pandas as pd

In [ ]:
type(DS_Salud)

In [ ]:
df = DS_Salud.toPandas()


In [ ]:
type(df)

In [ ]:
# Formato de variables ScorecardPy
dt_s = sc.var_filter(df, y="VENTA_SEGURO")


In [ ]:
sc.iv(df, y="VENTA_SEGURO")

In [ ]:
bins = sc.woebin(dt_s, y="VENTA_SEGURO")

In [ ]:
bins

In [ ]:
sc.woebin_plot(bins)